In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb.tensor as qtn


import register_ as reg
import algo_cooling 

import algo_boundary



In [2]:
reg.reg_complex_svd()
import torch
to_backend = algo_cooling.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=False)
to_backend_c = algo_cooling.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=False)


opt = algo_cooling.opt_(progbar=True)

In [3]:
Lx, Ly = (4, 4)
peps = qtn.PEPS.rand(Lx=Lx, Ly=Ly, bond_dim=3, seed=666, dtype="complex128")
peps_ = qtn.PEPS.rand(Lx=Lx, Ly=Ly, bond_dim=5, seed=666, dtype="complex128")

# peps.equalize_norms_(4.0)


pepsh = peps_.H
pepsh.add_tag('BRA')
peps.add_tag('KET')
norm = pepsh & peps
norm.apply_to_arrays(to_backend_c)

norm.apply_to_arrays(to_backend)


/Users/rezah/Documents/dmrg/algo_cooling.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x, dtype=dtype, device=device, requires_grad=requires_grad)


In [4]:
norm_ = norm.contract(all, optimize=opt)
norm = norm * (norm_**-1.)

In [5]:
norm_ = norm.contract(all, optimize=opt)
norm_

tensor(1.0000-2.6090e-15j, dtype=torch.complex128)

In [6]:

Bdympo = algo_boundary.BdyMPO(peps, opt=opt, chi=20, norm_tn=norm, 
                              to_backend=to_backend)
mpo_b = Bdympo.mps_b

In [7]:
norm_ = norm.contract(all, optimize=opt)
norm_

tensor(1.0000-2.6090e-15j, dtype=torch.complex128)

In [8]:
%%time
algo_boundary.comp_bdy(norm, mpo_b, Ly, opt=opt, eq_norms=False, str_r="_r_", str_l="_l_", 
                       n_iter=2, pbar=True, stop_grad_=True, fidel_=True, visual_=False)

dmrg:: 100%|██████████| 4/4 [00:01<00:00,  2.86it/s, F=0.483]

CPU times: user 1.07 s, sys: 399 ms, total: 1.47 s
Wall time: 1.41 s


tensor(0.6797-0.0241j, dtype=torch.complex128)

In [9]:
%%time
algo_boundary.comp_bdy(norm, mpo_b, Ly, opt=opt, eq_norms=False, str_r="_r", str_l="_l", 
                       n_iter=2, pbar=True, stop_grad_=True, fidel_=True, visual_=False)

dmrg:: 100%|██████████| 4/4 [00:00<00:00,  4.33it/s, F=0.628]

CPU times: user 814 ms, sys: 262 ms, total: 1.08 s
Wall time: 926 ms


tensor(0.6745+0.4177j, dtype=torch.complex128)

In [10]:
%%time
val_0 = norm.contract_boundary(max_bond=70, 
                                   mode="mps",#     "full-bond", #   'mps',
                                   sequence=["ymin", "ymax"],
                                   final_contract_opts={"optimize": opt}, 
                                   cutoff=1e-14,
                                   progbar=True,
                                   layer_tags=['KET', 'BRA'],
                                   max_separation=0,
                                  )
val_0

contracted boundary, Lx=4, Ly=1: : 3it [00:00, 16.01it/s]

CPU times: user 200 ms, sys: 102 ms, total: 301 ms
Wall time: 189 ms


tensor(1.0869+0.2776j, dtype=torch.complex128)

In [11]:
def apply_hyperoptimized_compressed(tn, copt, chi, output_inds=None, tree_gauge_distance=4, progbar=False, 
                                    cutoff=1.e-12, equalize_norms=False):
    
    tn.full_simplify_(seq='R', split_method='svd', inplace=True)
    
    tree = tn.contraction_tree(copt)
    tn_ = tn.copy()
    
    flops = tree.contraction_cost(log=10)
    peak = tree.peak_size(log=2)
    
    tn_.contract_compressed_(
        optimize=tree,
        output_inds=output_inds,
        max_bond=chi,
        tree_gauge_distance=tree_gauge_distance,
        equalize_norms=equalize_norms,
        cutoff=cutoff,
        progbar=progbar,
    )
    return tn_, (flops, peak)


In [12]:
%%time
chi = 70
import cotengra as ctg
copt = ctg.ReusableHyperCompressedOptimizer(
    chi,
    max_repeats=512,
    minimize="combo-compressed",
    progbar=True,
    # # save paths to disk:
    # directory="cash"
)

overlap, (flops, peak) = apply_hyperoptimized_compressed(norm, copt, chi, cutoff=1.e-12, progbar=True)
# main, exp=(overlap.contract(all), overlap.exponent)
val_0 = overlap^all 
print(val_0)


log2[SIZE]: 0.00/15.63: 100%|██████████| 31/31 [00:00<00:00, 455.71it/s]  

tensor(1.1402-0.0092j, dtype=torch.complex128)
CPU times: user 2.9 s, sys: 175 ms, total: 3.07 s
Wall time: 3.01 s
